# cxMate Service DEMO
By Ayato Shimada, Mitsuhiro Eto

This DEMO shows
1. detect communities using an __igraph's community detection algorithm__
2. __paint communities (nodes and edges)__ in different colors
3. perform layout using __graph-tool's sfdp algorithm__

---

In [6]:
# Tested on:
!python --version

Python 3.5.4 :: Continuum Analytics, Inc.


# Send CX to service using requests module

### Services are built on __a server__
You don't have to construct graph libraries in your local environment.  
It is very easy to use __python-igraph__ and __graph-tools__.

### In order to send CX
- requests : to send CX file to service in Python. (curl also can be used.)
- json : to convert *object* to a CX formatted string.


In [16]:
import requests
import json

In [17]:
url_community = 'http://localhost:80' # igraph's community detection service URL
url_layout = 'http://localhost:3000' # graph-tool's layout service URL
headers = {'Content-type': 'application/json'}

# Network used for DEMO

This DEMO uses `yeastHQSubnet.cx` as original network.
- 2924 nodes
- 6827 edges
<img src="example1.png" alt="Drawing" style="width: 500px;"/>

---

# 1. igraph community detection and color generator service
In order to detect communities, __igraph's community detection service__ can be used.  

### How to use the service on Jupyter Notebook
1. open the CX file using __*`open()`*__
2. set parameters in dictionary format. (About parameters, see the document of service.)
3. post the CX data to URL of service using __*`requests.post()`*__

In [20]:
data = open('./yeastHQSubnet.cx') # 1.
parameter = {'type': 'leading_eigenvector', 'clusters': 5, 'palette': 'husl'} # 2.
r = requests.post(url=url_community, headers=headers, data=data, params=parameter) # 3.

## What happened?

### Output contains
__graph with community membership__  + __color assignment for each group__.
- node1 : group 1, red
- node2 : group 1, red
- node3 : group 2, green
...

## You don't have to create your own color palette manually.

To save and look the output data, you can use __*`r.json()['data']`*__

__Note__  
- When you use this output as input of next service, you must use __*`json.dumps(r.json()['data'])`*__
- You must replace single quotation to double quotation in output file.

In [21]:
import re
with open('output1.cx', 'w') as f:
    # single quotation -> double quotation
    output = re.sub(string=str(r.json()['data']), pattern="'", repl='"')
    f.write(output)

# 3. graph-tool layout service
In order to perform layout algorithm, __graph-tool's layout algorithm service__ can be used.  

## C++ optimized parallel, community-structure-aware layout algorithms

You can use the community structure as a parameter for layout, and __result reflects its structure.__


You can use graph-tool's service __in the same way as igraph's service__.  
Both input and output of cxMate service are __CX__, NOT __igraph's object__, __graph-tool's object__ and so on.  
So, you __don't have to convert__ igraph object to graph-tools object.

<img src="service.png" alt="Drawing" style="width: 750px;"/>


### How to use the service on Jupyter Notebook
1. open the CX file using __*`json.dumps(r.json()['data'])`*__
2. set parameters in dictionary format. (About parameters, see the document of service.)
3. post the CX data to URL of service using __*`requests.post()`*__

In [22]:
data2 = json.dumps(r.json()['data']) # 1.
parameter = {'only-layout': False, 'groups': 'community'} # 2. 
r2 = requests.post(url=url_layout, headers=headers, data=data2, params=parameter) # 3.

## Save .cx file
To save and look the output data, you can use __*`r.json()['data']`*__


In [23]:
import re
with open('output2.cx', 'w') as f:
    # single quotation -> double quotation
    output = re.sub(string=str(r2.json()['data']), pattern="'", repl='"')
    f.write(output)

---

# Color Palette
If you want to change color of communities, you can do it __easily__.  
__Many color palettes of seaborn__ can be used.  (See http://seaborn.pydata.org/tutorial/color_palettes.html)

In [11]:
%matplotlib inline
import seaborn as sns, numpy as np
from ipywidgets import interact, FloatSlider

## Default Palette
Without setting parameter 'palette', 'husl' is used as color palette.

In [2]:
def show_husl(n):
    sns.palplot(sns.color_palette('husl', n))
print('palette: husl')
interact(show_husl, n=10);

palette: husl


A Jupyter Widget

## Other palettes

In [3]:
def show_pal0(palette):
    sns.palplot(sns.color_palette(palette, 24))
interact(show_pal0, palette='deep muted pastel bright dark colorblind'.split());

A Jupyter Widget

In [4]:
sns.choose_colorbrewer_palette('qualitative');

A Jupyter Widget

In [5]:
sns.choose_colorbrewer_palette('sequential');

A Jupyter Widget